In [16]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.preprocessing import image
tf.keras.utils.to_categorical

<function keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [18]:
idg = ImageDataGenerator(#width_shift_range=0.1,
                         #height_shift_range=0.1,
                         #zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True)
                         #rescale=1./255)

In [22]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [19]:
def create_new_model():
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Flatten())


    model.add(Dense(128))
    model.add(Activation('relu'))

    # Dropouts help reduce overfitting by randomly turning neurons off during training.
    # Here we say randomly turn off 50% of neurons.
    model.add(Dropout(0.5))

    # Last layer, not binary!
    model.add(Dense(3))
    model.add(Activation('sigmoid'))

    model.compile(loss='categorical_crossentropy', #not binary for the poses
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

In [23]:
train_data = pd.read_csv('classification_nfold.csv', dtype=str)
#data = pd.read_csv('./Hand_Annotations_2.csv',dtype=str,names=colnames
Y = train_data[['label']]

# kf = KFold(n_splits = 3)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)
image_shape = (150,150,3)

In [25]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'kfoldModels/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(564),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, target_size=image_shape[:2],
                               batch_size = 16,
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, target_size=image_shape[:2],
                            batch_size = 16,
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True)
    
  

    # CREATE NEW MODEL
    model = create_new_model()

    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=5,
                #steps_per_epoch=150,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    
    
   
    
    #PLOT HISTORY

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("kfoldModels/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    
    #calculate sensitivity and specificity 
    

    tf.keras.backend.clear_session()

    fold_var += 1

    

Found 507 validated image filenames belonging to 3 classes.
Found 57 validated image filenames belonging to 3 classes.
Epoch 1/5
32/32 [==============================] - ETA: 0s - loss: 32.6581 - accuracy: 0.4970
Epoch 1: val_accuracy improved from -inf to 0.70175, saving model to kfoldModels\model_1.h5
32/32 [==============================] - 37s 996ms/step - loss: 32.6581 - accuracy: 0.4970 - val_loss: 0.6605 - val_accuracy: 0.7018
Epoch 2/5
32/32 [==============================] - ETA: 0s - loss: 0.5867 - accuracy: 0.7850
Epoch 2: val_accuracy improved from 0.70175 to 0.80702, saving model to kfoldModels\model_1.h5
32/32 [==============================] - 31s 959ms/step - loss: 0.5867 - accuracy: 0.7850 - val_loss: 0.5973 - val_accuracy: 0.8070
Epoch 3/5
32/32 [==============================] - ETA: 0s - loss: 0.8642 - accuracy: 0.8205
Epoch 3: val_accuracy did not improve from 0.80702
32/32 [==============================] - 30s 937ms/step - loss: 0.8642 - accuracy: 0.8205 - val_lo

32/32 [==============================] - ETA: 0s - loss: 0.2733 - accuracy: 0.9094
Epoch 5: val_accuracy did not improve from 0.87500
4/4 [==============================] - 1s 210ms/step - loss: 0.4213 - accuracy: 0.8393
Found 508 validated image filenames belonging to 3 classes.
Found 56 validated image filenames belonging to 3 classes.
Epoch 1/5
32/32 [==============================] - ETA: 0s - loss: 33.8637 - accuracy: 0.5728
Epoch 1: val_accuracy improved from -inf to 0.78571, saving model to kfoldModels\model_6.h5
32/32 [==============================] - 35s 931ms/step - loss: 33.8637 - accuracy: 0.5728 - val_loss: 0.5898 - val_accuracy: 0.7857
Epoch 2/5
32/32 [==============================] - ETA: 0s - loss: 0.5569 - accuracy: 0.7972
Epoch 2: val_accuracy did not improve from 0.78571
32/32 [==============================] - 29s 890ms/step - loss: 0.5569 - accuracy: 0.7972 - val_loss: 0.5540 - val_accuracy: 0.7857
Epoch 3/5
32/32 [==============================] - ETA: 0s - loss

KeyboardInterrupt: 